In [3]:
%pip install -r requirements.txt

  Using cached pandas-2.3.2-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached numpy-2.3.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached ortools-9.14.6206-cp311-cp311-win_amd64.whl.metadata (3.1 kB)
  Using cached AST-0.0.2.tar.gz (19 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      Traceback (most recent call last):
        File "z:\Documents\Werk\MKB Datalab\Hoevelaken_backup\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "z:\Documents\Werk\MKB Datalab\Hoevelaken_backup\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "z:\Documents\Werk\MKB Datalab\Hoevelaken_backup\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\20192507\AppData\Local\Temp\pip-build-env-bitv7ah5\

In [4]:
import pandas as pd
import datetime as dt
import numpy as np
from ortools.sat.python import cp_model
from pandas import read_excel
import ast

ModuleNotFoundError: No module named 'pandas'

In [ ]:
info_shifts = {
    "Shifts": ['D1', 'D2', 'D3', 'D4', 'FM', 'GD', 'KOK', 'A1', 'A2', 'A3', 'GA', 'N'],
    "Begintijd": ['07:00', '07:30', '07:30', '08:45', '08:00', '08:00', '15:00', '15:00', '15:30', '18:30', '16:00', '22:45'],
    "Eindtijd": ['15:00', '15:00', '14:00', '12:45', '12:30', '14:00', '19:30', '23:00', '22:30', '22:30', '22:00', '07:00'],
    "Duur": [8, 7.5, 6.5, 4, 4.5, 6, 4.5, 8, 7, 4, 6, 8.25],
    "Maandag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Dinsdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Woensdag": [1,1,1,1,1,1,1,1,1,1,1,1],
    "Donderdag": [1,1,1,0,1,1,1,1,1,1,1,1],
    "Vrijdag": [1,1,1,1,0,1,1,1,1,1,1,1],
    "Zaterdag": [1,1,1,0,0,1,1,1,1,1,1,1],
    "Zondag": [1,1,1,1,0,1,1,1,1,0,1,1],
    "Deskundigheden": [[1,2], [2], [3], [3], [6], [4], [5], [2], [3], [3], [4], [2]]
}

for i in range(len(info_shifts['Shifts'])):
    info_shifts['Begintijd'][i] = dt.datetime.strptime(info_shifts['Begintijd'][i], '%H:%M').time()
    info_shifts['Eindtijd'][i] = dt.datetime.strptime(info_shifts['Eindtijd'][i], '%H:%M').time()
    
df_shifts = pd.DataFrame(info_shifts)

: 

In [ ]:
df_onb = pd.read_csv('data/Onbeschikbaarheid_hoevelaken.csv', sep=';')
df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')
df_onb

: 

In [ ]:
df_werknemers = pd.read_excel('data\medewerkers_compleet.xlsx')

#Convert deskundigheid to list of ints
lsts = []
for i in range(len(df_werknemers)):
    lst = []
    if df_werknemers['deskundigheid'][i] <10:
        lst = [int(df_werknemers['deskundigheid'][i])]
    else:
        lst = [int(x) for x in str(int(df_werknemers['deskundigheid'][i]))]
    
    lsts.append(lst)
     
df_werknemers['deskundigheid'] = lsts

# Convert contract geldig van, contract tm, and geboortedatum to datetime
df_werknemers['contract_vanaf'] = pd.to_datetime(df_werknemers['contact vanaf'], format='%d/%m/%Y')
df_werknemers['contract_tm'] = pd.to_datetime(df_werknemers['contract tm'], format='%d/%m/%Y')
#df_werknemers['geboortedatum'] = pd.to_datetime(df_werknemers['geboortedatum'], format='%d/%b', errors='coerce')
# Drop unnecessary columns
df_werknemers = df_werknemers.drop(columns=['contact vanaf', 'contract tm'])

df_werknemers

: 

In [ ]:
df_werknemers

# delete all rows where wensen = niet plannen
df_werknemers = df_werknemers[df_werknemers['wensen'] != 'niet plannen']
df_werknemers = df_werknemers.reset_index(drop=True)
df_werknemers

# If contract soort = oproep, set contracturen to 0
df_werknemers.loc[df_werknemers['contract soort'] == 'oproep', 'contracturen'] = 0


# If max_werkdgn_pw is NaN, set to 5
df_werknemers['max_werkdgn_pw'] = df_werknemers['max_werkdgn_pw'].fillna(5)

# If contracturen is -, set to 0
df_werknemers['contracturen'] = df_werknemers['contracturen'].replace('-', 0)

# DELETE REIJNDERS SINCE THIS IS AN ANOMALY
df_werknemers = df_werknemers[df_werknemers['medewerker_naam'] != 'Reijnders']
df_werknemers = df_werknemers.reset_index(drop=True)
df_werknemers

: 

In [ ]:
# Delete all rows where shifts = KOK or FM
df_shifts = df_shifts[~df_shifts['Shifts'].isin(['KOK', 'FM'])]

# Convert to long format: each row = one shift on one day
days = ['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag']
day_map = {day: i for i, day in enumerate(days)}  # Map day names to day indices (0–6)

shift_requirements = []

for idx, row in df_shifts.iterrows():
    for day in days:
        if row[day] == 1:  # If this shift needs to be filled on this day
            shift_requirements.append({
                "shift_name": row['Shifts'],
                "day_of_week": day_map[day],  # 0=Monday, etc.
                "start_time": row['Begintijd'],
                "end_time": row['Eindtijd'],
                "qualification": row['Deskundigheden'],
                "duration": row['Duur']
            })

all_shift_instances = []

for week in range(4):  # For 4 weeks
    week_num = week + 1  # Weeks are 1-indexed
    for row in shift_requirements:
        all_shift_instances.append({
            **row,
            "week": week_num,
            "absolute_day": week * 7 + row["day_of_week"],  # 0–27
        })

shift_instances_df = pd.DataFrame(all_shift_instances)

: 

## Preprocessing function

In [ ]:

def preprocess_data(df_shifts: pd.DataFrame, df_werknemers: pd.DataFrame, df_onb: pd.DataFrame, start_date: dt.date = dt.date(2025, 9, 1), num_weeks: int = 4):
    """
    Prepares shifts and workers dataframes for the OR-Tools scheduling model.
    Returns processed shifts, workers, and helper structures.
    """

    # --- Shifts ---
    # Create clean shifts dataframe
    # Delete all rows where shifts = KOK or FM
    df_shifts = df_shifts[~df_shifts['Shifts'].isin(['KOK', 'FM'])]

    # Convert to long format: each row = one shift on one day
    days = ['Maandag', 'Dinsdag', 'Woensdag', 'Donderdag', 'Vrijdag', 'Zaterdag', 'Zondag']
    day_map = {day: i for i, day in enumerate(days)}  # Map day names to day indices (0–6)

    shift_requirements = []

    for idx, row in df_shifts.iterrows():
        for day in days:
            if row[day] == 1:  # If this shift needs to be filled on this day
                shift_requirements.append({
                    "shift_name": row['Shifts'],
                    "day_of_week": day_map[day],  # 0=Monday, etc.
                    "start_time": row['Begintijd'],
                    "end_time": row['Eindtijd'],
                    "qualification": row['Deskundigheden'],
                    "duration": row['Duur']
                })

    all_shift_instances = []

    
    for week in range(num_weeks):
        for row in shift_requirements:
            # Compute shift_date based on start_date
            shift_date = start_date + dt.timedelta(days=week*7 + row["day_of_week"])
            all_shift_instances.append({
                **row,
                "week": week + 1,
                "absolute_day": week * 7 + row["day_of_week"],  # 0-based absolute day
                "shift_date": shift_date
            })

    shifts = pd.DataFrame(all_shift_instances)
    shifts['shift_id'] = shifts.index.astype(int)

    # Convert durations (hours → minutes, integers for CP-SAT)
    shifts['duration_min'] = (shifts['duration'].astype(float) * 60).round().astype(int)

    # Ensure int types
    shifts['week'] = shifts['week'].astype(int)
    shifts['day_of_week'] = shifts['day_of_week'].astype(int)

    # Day key for grouping
    shifts['day_key'] = list(zip(shifts['week'], shifts['day_of_week']))

    # add is_night column based on shift_name (e.g., 'N' is night)
    shifts['is_night'] = shifts['shift_name'].apply(lambda x: 1 if x == 'N' else 0)
    shifts['is_night'] = shifts['is_night'].astype(bool)
    shifts['shift_id'] = shifts.index.astype(int)
    
    shifts['qualification'] = shifts['qualification'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q)
    
    shifts = shifts[['shift_id', 'shift_name', 'shift_date', 'week', 'day_of_week', 'absolute_day', 'start_time', 'end_time', 'duration_min', 'qualification', 'is_night', 'day_key']]
    
    # --- Workers ---
    workers = df_werknemers.copy().reset_index(drop=True)
    workers['medewerker_id'] = workers['medewerker_id'].astype(str)
    # Change werknemers_id to delete every 0 at the start and everything and including the first -
    workers['medewerker_id'] = workers['medewerker_id'].apply(lambda x: x.lstrip('0').split('-')[0])
    
    # Convert contract geldig van, contract tm, and geboortedatum to datetime
    workers['contract_vanaf'] = pd.to_datetime(workers['contact vanaf'], format='%d/%m/%Y')
    workers['contract_tm'] = pd.to_datetime(workers['contract tm'], format='%d/%m/%Y')
    #If contract_tm is NaT, set to 31-12-2099
    workers['contract_tm'] = workers['contract_tm'].fillna(pd.Timestamp('2099-12-31'))
    
    # Drop unnecessary columns
    workers = workers.drop(columns=['contact vanaf', 'contract tm'])
    
    #Convert deskundigheid to list of ints
    lsts = []
    for i in range(len(workers)):
        lst = []
        if workers['deskundigheid'][i] <10:
            lst = [int(workers['deskundigheid'][i])]
        else:
            lst = [int(x) for x in str(int(workers['deskundigheid'][i]))]
        
        lsts.append(lst)
        
    workers['deskundigheid'] = lsts

    workers['deskundigheid'] = workers['deskundigheid'].apply(
        lambda q: ast.literal_eval(q) if isinstance(q, str) else q
    )
    # Delete all employees with deskundigheid = 5 or 6
    workers = workers[~workers['deskundigheid'].apply(lambda x: isinstance(x, (list, tuple)) and (5 in x or 6 in x or 7 in x))]
    
    # delete all rows where wensen = niet plannen
    workers = workers[workers['wensen'] != 'niet plannen']
    workers = workers.reset_index(drop=True)
    
    # If contract soort = oproep, set contracturen to 0
    workers.loc[workers['contract soort'] == 'oproep', 'contracturen'] = 0
    
    # Normalize contract details
    workers['max_days_per_week'] = workers['max_werkdgn_pw'].fillna(0).astype(int)
    workers['contract_hours'] = workers['contracturen'].astype(float)
    workers['contract_minutes'] = (workers['contract_hours'] * 60).round().astype(int)
    
    # Convert geboortedatum to leeftijd based on current day (round down if birthday not yet occurred this year)
    current_date = dt.datetime.now().date()

    workers['leeftijd'] = workers['geboortedatum'].apply(
        lambda x: current_date.year - x.year - ((current_date.month, current_date.day) < (x.month, x.day))
    )
    
    # Create IDs
    emp_ids = workers['medewerker_id'].tolist()
    #emp_ids = [str(e) for e in emp_ids]
    emp_index = {emp: i for i, emp in enumerate(emp_ids)}

    # --- onhbeschikbaarheid ---
    # Convert 'Datum beschikbaarheid' to datetime
    df_onb['Datum'] = pd.to_datetime(df_onb['Datum beschikbaarheid'], format='%d-%m-%Y').dt.date
    df_onb['Beschikbaarheid'] = df_onb['Beschikbaarheid'].fillna('Onbekend')
    df_onb['Beschikbaarheid_tijd_vanaf'] = pd.to_datetime(df_onb['Beschikbaarheid tijd vanaf'], format='%H:%M', errors='coerce').dt.time
    df_onb['Beschikbaarheid_tijd_tm'] = pd.to_datetime(df_onb['Beschikbaarheid tijd t/m'], format='%H:%M', errors='coerce').dt.time
    df_onb = df_onb[['Medewerker id', 'Datum', 'Beschikbaarheid', 'Beschikbaarheid_tijd_vanaf', 'Beschikbaarheid_tijd_tm']]
    df_onb['Medewerker id'] = df_onb['Medewerker id'].astype(str)

    # --- Helper dictionaries ---
    dur_min = shifts.set_index('shift_id')['duration_min'].to_dict()

    # Grouping by week/day
    shifts_by_week = {}
    shifts_by_day = {}
    for _, r in shifts.iterrows():
        s = int(r['shift_id'])
        w = int(r['week'])
        day_k = r['day_key']
        shifts_by_week.setdefault(w, []).append(s)
        shifts_by_day.setdefault(day_k, []).append(s)

    weeks = sorted(shifts['week'].unique().tolist())
    
    night_shifts = shifts.loc[shifts['is_night'], 'shift_id'].tolist()
    night_shifts_by_week = {
        w: shifts.loc[(shifts['week'] == w) & (shifts['is_night']), 'shift_id'].tolist() for w in weeks}

    return {
        "shifts": shifts,
        "workers": workers,
        "onb": df_onb,
        "emp_ids": emp_ids,
        "emp_index": emp_index,
        "dur_min": dur_min,
        "shifts_by_week": shifts_by_week,
        "shifts_by_day": shifts_by_day,
        "weeks": weeks,
        "night_shifts": night_shifts,
        "night_shifts_by_week": night_shifts_by_week
    }

preprocessed_data = preprocess_data(df_shifts, df_werknemers, df_onb)
preprocessed_data['shifts']
preprocessed_data['shifts']

: 

# Auto-Rooster

In [ ]:
def solve_schedule_max_cov_uncovered_night_CAO_onb_qual_wensen(data, time_limit_s=60):
    shifts = data['shifts']
    workers = data['workers']
    onb = data['onb']
    emp_ids = data['emp_ids']
    dur_min = data['dur_min']
    shifts_by_week = data['shifts_by_week']
    shifts_by_day = data['shifts_by_day']
    weeks = data['weeks']
    night_shifts = data['night_shifts']
    night_shifts_by_week = data['night_shifts_by_week']
    
    CAO_7_4_exempt = set(map(str, [2097, 602859])) # Hennie and Teuna

    # --- helper maps once 
    days_list = sorted(shifts['absolute_day'].unique().tolist())
    night_shifts_by_day = shifts[shifts['is_night']].groupby('absolute_day')['shift_id'].apply(list).to_dict()
    
    model = cp_model.CpModel()
    
    ### Variables ###
    # Decision variables
    x = {(s, emp): model.NewBoolVar(f"x_s{s}_e{emp}")
         for s in shifts['shift_id'] for emp in emp_ids}

    # Shift not covered variable to allow max coverage
    u = {s: model.NewBoolVar(f"uncovered_s{s}") for s in shifts['shift_id']}

    # Night-indicator BoolVars per employee per day: n_emp_day = 1 iff emp works night shift that day
    n_emp_day = {}  # (emp, day) -> BoolVar
    for emp in emp_ids:
        for day in days_list:
            b = model.NewBoolVar(f"n_emp{emp}_day{day}")
            n_emp_day[(emp, day)] = b
            night_ids = night_shifts_by_day.get(day, [])
            if night_ids:
                # If b == 1  => sum(x[night_ids]) >= 1
                model.Add(sum(x[(s, emp)] for s in night_ids) >= 1).OnlyEnforceIf(b)
                # If b == 0  => sum(x[night_ids]) == 0
                model.Add(sum(x[(s, emp)] for s in night_ids) == 0).OnlyEnforceIf(b.Not())
            else:
                # no night shifts possible that day -> force b = 0
                model.Add(b == 0)
                
    ### Constraints ###
    ## Coverage and basic contract constraints
    # 1) Coverage constraint: sum of x + uncovered == 1
    for s in shifts['shift_id']:
        model.Add(sum(x[(s, emp)] for emp in emp_ids) + u[s] == 1)

    # 2) At most one shift per employee per day
    for emp in emp_ids:
        for day_k, s_list in shifts_by_day.items():
            model.AddAtMostOne(x[(s, emp)] for s in s_list)
    
    # 3) After night shift, no day or evening shift the next day. but night shifts are allowed
    for emp in emp_ids:
        for day in shifts['absolute_day'].unique():
            # Night shifts on this day
            night_shifts_today = shifts.loc[(shifts['absolute_day'] == day) & (shifts['is_night']), 'shift_id'].tolist()
            if not night_shifts_today:
                continue
            
            # allow next day night shifts, but no day or evening shifts
            next_day_shifts = shifts.loc[(shifts['absolute_day'] == day + 1) & (~shifts['is_night']), 'shift_id'].tolist()
            if not next_day_shifts:
                continue
            
            # Create a combined constraint:
            # If any night shift is worked on day D, no shift can be worked on day D+1
            model.Add(
                sum(x[(ns, emp)] for ns in night_shifts_today) +
                sum(x[(bs, emp)] for bs in next_day_shifts)
                <= len(night_shifts_today)
            )

    # 4) Max work days per week
    for emp in emp_ids:
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(x[(s, emp)] for s in s_list) <= max_days)
    
    # 5) Contract hours per week
    for emp in emp_ids:
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(dur_min[s] * x[(s, emp)] for s in s_list) <= cap_minutes)
    
    # 5) Contract hours per week
    for emp in emp_ids:
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        for w in weeks:
            s_list = shifts_by_week.get(w, [])
            if s_list:
                model.Add(sum(dur_min[s] * x[(s, emp)] for s in s_list) <= cap_minutes)
    
    # 6) Respect unavailable times
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in emp_ids:
            continue

        date_unb = r['Datum']
        besch = r['Beschikbaarheid'].lower()
        
        # If fully available, skip
        if besch in ['beschikbaar']:
            continue

        # Get unavailability times for that day
        start_unb = r.get('Beschikbaarheid_tijd_vanaf')
        end_unb = r.get('Beschikbaarheid_tijd_tm')
        
        for _, sr in shifts.iterrows():
            shift_date = sr['shift_date']
            if shift_date != date_unb:
                continue
            
            # Check if the shift overlaps with unavailability
            shift_start = sr['start_time']
            shift_end = sr['end_time']
            
            # Overlap condition
            overlap = not (shift_end <= start_unb or shift_start >= end_unb)
            if overlap:
                s = int(sr['shift_id'])
                model.Add(x[(s, emp)] == 0)
    
    ## CAO night shift constraints
    # 7.1) Max 5 consecutive nights
    for emp in emp_ids:
        night_shift_ids = [s for s in night_shifts if s in x]  # Only include available ones
    # Sort night shifts by absolute_day to detect sequences
    nights_sorted = sorted(night_shift_ids, key=lambda s: shifts.loc[shifts['shift_id']==s, 'absolute_day'].iloc[0])
    for i in range(len(nights_sorted) - 5):
        # Window of 6 consecutive nights
        window = nights_sorted[i:i+6]
        model.Add(sum(x[(s, emp)] for s in window) <= 5)
    
    # 7.2) After ≥3 consecutive nights → 46h rest (next 2 days off)
    # Detect a block end: days (d-2, d-1, d) are all nights AND (d+1) is NOT a night => that's an end-of-block
    # For each such possible end day `d`, forbid any shift on d+1 and d+2 when the condition holds.
    for emp in emp_ids:
        for idx in range(2, len(days_list)):             # idx corresponds to day d at days_list[idx]
            d2 = days_list[idx]   # the candidate "end" day (third of triple)
            d1 = days_list[idx-1]
            d0 = days_list[idx-2]
            # condition literals: n(emp,d0) & n(emp,d1) & n(emp,d2) & not n(emp,d2+1)
            # require d2+1 to exist in planning horizon to test next-day not-night; if it doesn't exist, skip enforcement
            if (d2 + 1) not in days_list:
                continue
            # blocked days are d2+1 and d2+2 (d2+2 may or may not exist; we will handle existing ones)
            blocked_days = [d2 + 1, d2 + 2]
            # build literal list for OnlyEnforceIf: [n_d0, n_d1, n_d2, n_{d2+1}.Not()]
            cond_literals = [
                n_emp_day[(emp, d0)],
                n_emp_day[(emp, d1)],
                n_emp_day[(emp, d2)],
                n_emp_day[(emp, d2 + 1)].Not()
            ]

            # collect blocked shift ids that actually exist in our horizon
            blocked_shift_ids = []
            for bd in blocked_days:
                blocked_shift_ids.extend(shifts_by_day.get(bd, []))

            if not blocked_shift_ids:
                continue

            # enforce: when the condition holds (i.e. the triple is the end of a night-block),
            # then every blocked shift must be 0 (no assignment).
            for bs in blocked_shift_ids:
                model.Add(x[(bs, emp)] == 0).OnlyEnforceIf(cond_literals)

    # 7.3) Max 35 nights per 13 weeks
    for emp in emp_ids:
        for start_week in range(min(weeks), max(weeks)-12):
            window_weeks = list(range(start_week, start_week+13))
            window_shifts = [s for w in window_weeks for s in night_shifts_by_week.get(w, [])]
            if window_shifts:
                model.Add(sum(x[(s, emp)] for s in window_shifts) <= 35)
    
    # 7.4) Employee older that 55: no night shifts
    for emp in emp_ids:
        # Exclude employees who have chosen to work nights
        if emp in CAO_7_4_exempt:
            continue
        
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55:
            for s in night_shifts:
                model.Add(x[(s, emp)] == 0)
    
    # 8) Qualification match: only assign employees to shifts they are qualified for    
    for _, shift_row in shifts.iterrows():
        sid = int(shift_row['shift_id'])
        required_quals = shift_row['qualification']  # could be int or list

        # Make sure required_quals is a list
        if not isinstance(required_quals, list):
            required_quals = [required_quals]

        for emp in emp_ids:
            emp_quals = workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
            if not any(q in emp_quals for q in required_quals):
                # Employee does NOT have any of the required qualifications → forbid assignment
                model.Add(x[(sid, emp)] == 0)
    
    ## Wensen constraints
    
    # 9.1) Hennie van den Brink (2097): Alleen nachtdiensten
    emp_vdBrink = '2097'
    if emp_vdBrink in emp_ids:
        for s in shifts['shift_id']:
            if s not in night_shifts:
                model.Add(x[(s, emp_vdBrink)] == 0)
                
    ### Objective ###
    # Objective: minimize uncovered shifts
    penalty_uncovered = 1  # weight for uncovered shifts

    model.Minimize(
        penalty_uncovered * sum(u.values())
        )

    ### Solve ###
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_s
    solver.parameters.num_search_workers = 8
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        assignments = []
        uncovered = []
    
        for _, r in shifts.iterrows():
            sid = int(r['shift_id'])
            assigned = False
            
            for emp in emp_ids:
                if solver.Value(x[(sid, emp)]) == 1:
                    assignments.append({
                        'shift_id': sid,
                        'shift_name': r['shift_name'],
                        'start_time': r['start_time'],
                        'end_time': r['end_time'],
                        'shift_date': r['shift_date'],
                        'is_night': r['is_night'],
                        'week': int(r['week']),
                        'day_of_week': int(r['day_of_week']),
                        'absolute_day': int(r['absolute_day']),
                        'duration_min': int(r['duration_min']),
                        'employee_id': emp,
                        'employee_name': workers.loc[workers['medewerker_id'] == emp, 'medewerker_naam'].iloc[0],
                        'qualification': r['qualification'],
                        'deskundigheid': workers.loc[workers['medewerker_id'] == emp, 'deskundigheid'].iloc[0]
                    })
                    assigned = True
                    break
            
            # If no employee assigned, mark as uncovered with missing employee info
            if not assigned:
                uncovered.append(sid)
                assignments.append({
                    'shift_id': sid,
                    'shift_name': r['shift_name'],
                    'is_night': r['is_night'],
                    'week': int(r['week']),
                    'day_of_week': int(r['day_of_week']),
                    'absolute_day': int(r['absolute_day']),
                    'start_time': r['start_time'],
                    'end_time': r['end_time'],
                    'duration_min': int(r['duration_min']),
                    'employee_id': None,
                    'employee_name': None,
                    'qualification': r['qualification'],
                    'deskundigheid': None
                })
        
        # Calculate penalties explicitly
        uncovered_penalty = len(uncovered) * penalty_uncovered
        
        print("=== Solution Summary ===")
        print(f"  Uncovered shifts: {len(uncovered)} (penalty {uncovered_penalty})")
        
        return {
            "assignments_df": pd.DataFrame(assignments),
            "uncovered_shifts": uncovered,
            "objective_value": solver.ObjectiveValue(),
            "solver_status": solver.StatusName(status)
        }
    else:
        print("No solution found:", solver.StatusName(status))
        return None

: 

In [ ]:
result = solve_schedule_max_cov_uncovered_night_CAO_onb_qual_wensen(preprocessed_data, time_limit_s=120)
result['assignments_df']

# Validation function

In [ ]:
def validate_solution_max_cov_uncovered_night_CAO_onb_qual_wensen(assignments_df, shifts, workers, dur_min, onb, night_shifts, night_shifts_by_week):
    errors = []

    # delete all rows in assignments_df where employee_id is NaN
    assignments_df = assignments_df.dropna(subset=['employee_id'])
    
    # Helper: quick lookup
    shifts_lookup = shifts.set_index('shift_id')

    CAO_7_4_exempt = set(map(str, [2097, 602859])) # Hennie and Teuna


    ## 1) Coverage: each shift has <= 1 assigned employee
    for sid, group in assignments_df.groupby("shift_id"):
        if len(group) > 1:
            errors.append(f"Shift {sid} assigned to multiple employees: {group['employee_id'].tolist()}")

    ## 2) At most one shift per employee per day
    for (emp, day), group in assignments_df.groupby(["employee_id", "absolute_day"]):
        if len(group) > 1:
            errors.append(f"Employee {emp} works {len(group)} shifts on day {day}: {group['shift_id'].tolist()}")

    ## 3) After night shift, no day/evening shift next day
    for emp, group in assignments_df.groupby("employee_id"):
        nights = group[group["is_night"] == True]
        for _, ns in nights.iterrows():
            next_day = ns['absolute_day'] + 1
            next_day_shifts = group[group['absolute_day'] == next_day]
            #if next_day_shifts is isnight shifts, it's ok
            next_day_shifts = next_day_shifts[next_day_shifts['is_night'] == False]
            
            # Any next day assignment is an error
            if len(next_day_shifts) > 0:
                errors.append(f"Employee {emp} has day/evening shift(s) on day {next_day} after a night shift on day {ns['absolute_day']}.")

    ## 4) Max work days per week
    for emp, group in assignments_df.groupby("employee_id"):
        max_days = int(workers.loc[workers['medewerker_id'] == emp, 'max_days_per_week'].iloc[0])
        for week, week_group in group.groupby("week"):
            worked_days = week_group['absolute_day'].nunique()
            if worked_days > max_days:
                errors.append(f"Employee {emp} exceeds max work days in week {week}: {worked_days} > {max_days}")

    ## 5) Contract hours per week
    for emp, group in assignments_df.groupby("employee_id"):
        cap_minutes = int(workers.loc[workers['medewerker_id'] == emp, 'contract_minutes'].iloc[0])
        for week, week_group in group.groupby("week"):
            total_minutes = week_group['duration_min'].sum()
            if total_minutes > cap_minutes:
                errors.append(f"Employee {emp} exceeds contract minutes in week {week}: {total_minutes} > {cap_minutes}")

    ## 6) No shifts overlapping with unavailable times
    for _, r in onb.iterrows():
        emp = r['Medewerker id']
        if emp not in workers['medewerker_id'].values:
            continue

        besch = r['Beschikbaarheid'].lower()
        if besch == 'beschikbaar':
            continue

        date_unb = r['Datum']
        start_unb = r.get('Beschikbaarheid_tijd_vanaf')
        end_unb = r.get('Beschikbaarheid_tijd_tm')

        emp_shifts = assignments_df[assignments_df["employee_id"] == emp]
        for _, sr in emp_shifts.iterrows():
            shift_date = sr['shift_date']
            if shift_date != date_unb:
                continue

            shift_start = sr['start_time']
            shift_end = sr['end_time']

            # If no specific times are given → treat as fully unavailable
            if pd.isna(start_unb) or pd.isna(end_unb):
                errors.append(
                    f"Employee {emp} scheduled on fully unavailable day {date_unb}, shift {sr['shift_id']}."
                )
                continue

            # Check overlap
            overlap = not (shift_end <= start_unb or shift_start >= end_unb)
            if overlap:
                errors.append(
                    f"Employee {emp} scheduled during unavailable time on {date_unb}, "
                    f"shift {sr['shift_id']} ({shift_start}–{shift_end} overlaps with {start_unb}–{end_unb})."
                )

    ## 7.1) Max 5 consecutive nights
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night'] == True].sort_values("absolute_day")
        days = emp_nights['absolute_day'].tolist()
        for i in range(len(days)-5):
            window = days[i:i+6]
            if window[-1] - window[0] == 5:  # 6 consecutive days
                errors.append(f"Employee {emp} works >5 consecutive nights (days {window}).")

    ## 7.2) After ≥3 consecutive nights → no shifts next 2 days
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night'] == True].sort_values("absolute_day")
        days = emp_nights['absolute_day'].tolist()
        for i in range(len(days)-2):
            window = days[i:i+3]
            if window[-1] - window[0] == 2:  # 3 consecutive nights
                blocked_days = [window[-1]+1, window[-1]+2]
                next_shifts = group[group['absolute_day'].isin(blocked_days)]
                if len(next_shifts) > 0:
                    errors.append(f"Employee {emp} has shifts {next_shifts['shift_id'].tolist()} within 46h rest after 3 nights ({window}).")

    ## 7.3) Max 35 nights per 13 weeks
    for emp, group in assignments_df.groupby("employee_id"):
        emp_nights = group[group['is_night'] == True]
        for start_week in range(shifts['week'].min(), shifts['week'].max()-12):
            window = emp_nights[(emp_nights['week'] >= start_week) & (emp_nights['week'] < start_week+13)]
            if len(window) > 35:
                errors.append(f"Employee {emp} has {len(window)} nights in weeks {start_week}-{start_week+12} (> 35).")

    ## 7.4) Age > 55: no night shifts
    for emp, group in assignments_df.groupby("employee_id"):
        if emp in CAO_7_4_exempt:
            continue
        
        leeftijd = int(workers.loc[workers['medewerker_id'] == emp, 'leeftijd'].iloc[0])
        if leeftijd > 55:
            if group['is_night'].any():
                errors.append(f"Employee {emp} (age {leeftijd}) assigned to night shifts: {group[group['is_night']==True]['shift_id'].tolist()}")

    ## 8) Qualification match
    for _, row in assignments_df.iterrows():
        sid = row['shift_id']
        required_quals = shifts_lookup.loc[sid, 'qualification']
        if not isinstance(required_quals, list):
            required_quals = [required_quals]
        emp_quals = workers.loc[workers['medewerker_id'] == row['employee_id'], 'deskundigheid'].iloc[0]
        if not any(q in emp_quals for q in required_quals):
            errors.append(f"Employee {row['employee_id']} lacks qualification {required_quals} for shift {sid}.")

        ## 9) Employee-specific hard constraints
    for _, row in assignments_df.iterrows():
        emp = row['employee_id']
        sid = row['shift_id']
        shift = shifts_lookup.loc[sid]
        day_of_week = int(shift['day_of_week'])
        is_night = shift['is_night']
        print(emp, sid, day_of_week, is_night)

        # 9.1) Hennie van den Brink (2097): Alleen nachtdiensten
        if emp == 2097 and not is_night:
            errors.append(
                f"Employee 2097 (Hennie van den Brink) assigned to non-night shift {sid} (day {day_of_week})."
            )

        # 9.2) Romy van Dijkhuizen (2653): Alleen weekenden (day_of_week 5=Sat, 6=Sun)
        if emp == 2653 and day_of_week not in [5, 6]:
            errors.append(
                f"Employee 2653 (Romy van Dijkhuizen) assigned to weekday shift {sid} (day {day_of_week})."
            )

        # 9.3) Jasmijn van den Berg (3888): Alleen vrijdag nacht en weekenden
        if emp == 3888:
            allowed = (day_of_week == 4 and is_night) or day_of_week in [5, 6]
            if not allowed:
                errors.append(
                    f"Employee 3888 (Jasmijn van den Berg) assigned to disallowed shift {sid} "
                    f"(day {day_of_week}, night={is_night})."
                )
    
    if errors:
        print(f"=== VALIDATION FAILED: {len(errors)} issue(s) ===")
        for e in errors:
            print("❌", e)
    else:
        print("✅ All constraints satisfied!")

In [ ]:
validate_solution_max_cov_uncovered_night_CAO_onb_qual_wensen(result["assignments_df"], 
                                                     preprocessed_data['shifts'],
                                                        preprocessed_data['workers'],
                                                        preprocessed_data['dur_min'],
                                                        preprocessed_data['onb'],
                                                        preprocessed_data['night_shifts'],
                                                        preprocessed_data['night_shifts_by_week'])